In [1]:
import pandas as pd
import requests
# pip install beautifulsoup4
from bs4 import BeautifulSoup
import math
import numpy as np
# pip install cdifflib
import difflib

In [2]:
# Set required dataset here.
data = pd.read_csv("data/5000_accounts_climate.csv")
data

,id,name,username,location,url,description,verified,followers,friends,favourites_count,statuses_count,created_at,default_profile,default_profile_image
0,1098803589609189376,💧The Cranky Croation,JohnSarich2,NaN,NaN,My First ever vote was for Gough Whitlam. Left...,False,430,291,14866,6039,2019-02-22 04:36:05,True,False
1,191393940,FranS #RejoinEU #ElectoralReform 🇬🇧🇪🇺🇩🇪,FranS199,Earth,NaN,"Love good food, good wine, great company. Hate...",False,4643,4904,257039,41696,2010-09-16 10:30:21,True,False
2,1185167241819676673,mbnvcxz,mbnvcxz2,NaN,NaN,NaN,False,0,3,1,8,2019-10-18 12:14:42,True,True
3,1220868357647368193,Noles-4-Life-In-SC,Noles4LifeInSC,NaN,NaN,Florida native living in Upstate South Carolin...,False,628,630,1852,1862,2020-01-25 00:39:20,True,False
4,347142932,Life Cycle UK,LifeCycleUKteam,Bristol,http://t.co/oCrHfe7xAg,Life Cycle UK is a Bristol-based charity that ...,False,3105,1645,3039,5349,2011-08-02 10:21:33,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958,998260558104276992,Illinois is broke and corrupt,IsIllinois,Illinois,NaN,Illinois is broke and corrupt. Every day it’s ...,False,1966,323,116697,44973,2018-05-20 17:53:58,True,False
4959,1056877279928860672,Alberto Farias,Alberto42456937,NaN,NaN,NaN,False,67,1243,1474,2193,2018-10-29 11:55:54,True,False
4960,1108097903728300032,Maude Nificent,maudenificent,NaN,NaN,"Baby-boomer, she, of yt settler stock; occupyi...",False,637,1581,21009,4845,2019-03-19 20:08:22,True,False
4961,214352450,K. Benzene,doctorbenzene,"Alberta, Canada",NaN,"Concerned citizen, Albertan originally from Sa...",False,1813,1282,59976,23726,2010-11-11 05:44:51,False,False


In [3]:
# WEBSITE CRAWLER FOR TITLES

# FEATURE 1 (url_titles)
# FEATURE 2 (username-urltitle_similarity)
# FEATURE 3 (username_in_urltitle)



titles = []
usern_title_sim = []
usern_foundin_urlTitle = []

for idx, row in data.iterrows():
    
    if not pd.isnull(data.iloc[idx,4]):
       #print(idx)
        try:
            # Get the url
            r = requests.get(data.iloc[idx,4])
            # Get content of websites
            soup = BeautifulSoup(r.content)
            # Remove html tokens and append.
            titles.append(str(soup.title).replace("<title>" , "").replace("</title>", "").strip("|"))
            # If site allows access... else fill columns with nan.
            if titles[idx].strip() != "403 Forbidden":
                usern_title_sim.append(difflib.SequenceMatcher(None, data["username"][idx].replace("_", "").lower(),
                                                               titles[idx].replace("'", "").replace("_", "").lower()).ratio())
                usern_foundin_urlTitle.append(data["username"][idx].lower().replace("_", "") 
                                          in titles[idx].replace(" ", "").replace("'", "").replace("_", "").lower())
            else:
                usern_title_sim.append(math.nan)
                usern_foundin_urlTitle.append(math.nan)
        # Handle exception if trying to enter website exceeds limit of tries and return "Failed connection".       
        except Exception:
            titles.append("Failed connection")
            usern_title_sim.append(math.nan)
            usern_foundin_urlTitle.append(math.nan)
    else:
        titles.append(math.nan)
        usern_title_sim.append(math.nan)
        usern_foundin_urlTitle.append(math.nan)

# Add features to dataset
data["url_titles"] = titles
data["username_urltitle_simimlarity"] = usern_title_sim
data["username_in_urltitle"] = usern_foundin_urlTitle

In [4]:
# Similarity of username to screen name.
# FEATURE 4 (username_name_similarity)

usern_name_sim = []

for idx, row in data.iterrows():
    usern_name_sim.append(difflib.SequenceMatcher(None, data["username"][idx].replace("_", "").lower()
                                                  , data["name"][idx]
                                                  .replace("'", "").replace("_", "").replace("-", "").lower()).ratio())
# Add feature to dataset
data["username_name_similarity"] = usern_name_sim

In [5]:
# username also featured in bio (the username must be an exact sub-set o th e description to match.)
# FEATURE 5 (username_in_bio)

username_found = []

for idx, row in data.iterrows():
    if data["username"][idx].replace("_", "").lower() in str(data["description"][idx]).lower().replace("'", "").replace("_", "").replace("-", ""):
        username_found.append(True)
    else:
        username_found.append(False)

data["username_in_bio"] = username_found

In [6]:
# lower user id, early twitter users 
# FEATURE 6 (lower_userid)

username_lower = []
# Find out what is considered a low id and set it as threshold.
id_threshold = 999999999 # 9 x9

for idx, row in data.iterrows():
    if data["id"][idx] <= id_threshold:
        username_lower.append(True)
    else:    
        username_lower.append(False)

# Add feature to dataset
data["lower_userid"] = username_lower

In [7]:
# followers to friends ratio (the bigger the ratio, the more the chance to be a business)
# FEATURE 7 (followers_friends_ratio)

# Add feature to dataset
data["followers_friends_ratio"] = data["followers"] / data["friends"]

In [8]:
# Bio Sentiment.
# FEATURE 8 (bio_sentiment)

# pip install afinn
from afinn import Afinn

sentiment_list = []
af = Afinn()

for idx, row in data.iterrows():
    if af.score(str(data["description"][idx])) > 0:
        sentiment_list.append("positive")
    elif af.score(str(data["description"][idx])) < 0:
        sentiment_list.append("negative")
    else:
        sentiment_list.append("neutral")
        

# Add feature to dataset
data["bio_sentiment"] = sentiment_list

In [9]:
# Show dataset with updated features.
final_df = data[["id",
                 "url_titles",
                 "username_urltitle_simimlarity",
                 "username_in_urltitle",
                 "username_name_similarity",
                 "username_in_bio",
                 "lower_userid",
                 "followers_friends_ratio",
                 "bio_sentiment"]].copy()

In [10]:
final_df

,id,url_titles,username_urltitle_simimlarity,username_in_urltitle,username_name_similarity,username_in_bio,lower_userid,followers_friends_ratio,bio_sentiment
0,1098803589609189376,NaN,NaN,NaN,0.129032,False,False,1.477663,negative
1,191393940,NaN,NaN,NaN,0.212766,False,True,0.946778,positive
2,1185167241819676673,NaN,NaN,NaN,0.933333,False,False,0.000000,neutral
3,1220868357647368193,NaN,NaN,NaN,1.000000,False,False,0.996825,neutral
4,347142932,Life Cycle UK,0.758621,False,0.785714,False,True,1.887538,positive
...,...,...,...,...,...,...,...,...,...
4958,998260558104276992,NaN,NaN,NaN,0.410256,False,False,6.086687,negative
4959,1056877279928860672,NaN,NaN,NaN,0.482759,False,False,0.053902,neutral
4960,1108097903728300032,NaN,NaN,NaN,0.962963,False,False,0.402910,negative
4961,214352450,NaN,NaN,NaN,0.608696,False,True,1.414197,negative


In [11]:
final_df.to_csv("business_account_features.csv")